# Run Lifetime Analysis Trade Studies

## How it Works 

* Set up the parameters needed to generate the trade study
    * High level trade study options
    * Orbital elements
    * Spacecraft properties
    * Force model options 
    * Lifetime tool specific settings
    * Define what and how things are varied: LatinHyperCube, GridSearch, Perturb State, LifeTimeVariations vs HPOP 
    * The trade study will be saved in a subfolder called TradeStudyFiles
* Generate different runs according to the trade study and save to a csv in a subfolder called Results
    * Generate a new trade study with blank results if no results are found
    * If results are found try to merge trade study subsets into one csv file and delete subset files
    * Read in the csv of results
* Run the trade study
    * The csv will be split into subsets according to the numCores variable to be run on different STK instances 
    * Any row without a result (a lifetime estimate or HPOP estimate) will be run. A row with an existing result will be skipped.
    * Once all rows have been executed on a trade study subset the STK process will shut down and a 'done' message will appear.
    * Rerun this script once the trade study is complete to merge results into one csv file and delete any subset csvs and verify all runs were completed
    * If all entries have a result then STK will not be started and a message will be printed informing you the trade study has already been completed.

You will need to install **pyDOE2** and **poliastro** in order to build the set of runs

A Results and TradeStudyFiles subfolder will be created if they do not already exist.


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
import os
import time
from functools import reduce
from comtypes.client import CreateObject
from comtypes.client import GetActiveObject
from comtypes.gen import STKObjects
from comtypes.gen import STKUtil
from comtypes.gen import AgSTKVgtLib
from pyDOE2 import fullfact,lhs # Will need to install pyDOE2: pip install pyDOE2    OR    conda install -c conda-forge pydoe2
from poliastro.core.elements import rv2coe,coe2rv # Will need to install poliastro: conda install -c conda-forge poliastro
from poliastro.constants import GM_earth
import asyncio
from threading import Thread
import concurrent.futures
import logging
import sys
import pythoncom
from IPython.display import Image
from contextlib import suppress
from LifeTimeLib import * 
import pickle
%config Completer.use_jedi = False
# Create sub directory to store files
if not os.path.exists(os.getcwd()+'\\Results'):
    os.mkdir(os.getcwd()+'\\Results')
if not os.path.exists(os.getcwd()+'\\TradeStudyFiles'):
    os.mkdir(os.getcwd()+'\\TradeStudyFiles')

## Create a new default trade study 

In [2]:
tradeStudy = TradeStudy()
tradeStudy.properties()

{'fileName': 'LifeTimeAnalysis.csv',
 'numCores': 2,
 'runHPOP': False,
 'maxDur': 100,
 'decayAlt': 65,
 'epoch': 20001,
 'a': 6778,
 'e': 0,
 'i': 45,
 'AoP': 0,
 'RAAN': 0,
 'TA': 0,
 'Cd': 2.2,
 'Cr': 1.0,
 'DragArea': 13.65,
 'SunArea': 15.43,
 'Mass': 1000,
 'AtmDen': 'Jacchia 1971',
 'SolFlxFile': 'SolFlx_CSSI.dat',
 'SigLvl': 0,
 'OrbPerCal': 10,
 'GaussQuad': 2,
 'SecondOrderOblateness': 'Off',
 'howToVary': 'LatinHyperCube',
 'numberOfRuns': 1,
 'varyCols': [],
 'varyValues': [],
 'setSunAreaEqualToDragArea': True}

**Modify Trade Study Settings**

These settings will be used for all runs if the variable is not not varied.

Units: Epoch: days, Distance: km, Velocity: km/s, Area: m^2, Mass: kg

It is recommended to start out with only 2-4 cores and look at the memory usage before moving to a higher number of cores.


In [3]:
# High level inputs
tradeStudy.fileName = 'LifeTimeExample.csv' # Where to save runs
tradeStudy.runHPOP = False # Run HPOP in addition to the Lifetime Tool
tradeStudy.numCores = 2 # How many instances of STK to use

# Orbit Epoch and State
tradeStudy.epoch = 19253.16666667 # yyddd.ddd format # 19253.16666667 is 10 Sep 2019 04:00:00.000
tradeStudy.a = 6778
tradeStudy.e = 0
tradeStudy.i = 45
tradeStudy.AoP = 0
tradeStudy.RAAN = 0
tradeStudy.TA = 0

# Spacecraft Properties
tradeStudy.Cd = 2.2
tradeStudy.Cd = 1.0
tradeStudy.DragArea = 13.65
tradeStudy.SunArea = 15.43
tradeStudy.Mass = 1000
    
# Lifetime Tool Settings
tradeStudy.maxDur = 100
tradeStudy.decayAlt = 65
tradeStudy.AtmDen = 'Jacchia 1971'
tradeStudy.SolFlxFile = 'SolFlx_CSSI.dat'
tradeStudy.SigLvl = 0
tradeStudy.OrbPerCal = 10
tradeStudy.GaussQuad = 2
tradeStudy.SecondOrderOblateness = 'Off'

**Define What is Varied**

Columns which can be varied, which will override any default settings:

'epoch', 'a', 'e', 'i', 'RAAN', 'AoP', 'TA', 'Rp', 'Ra',
'x', 'y', 'z','Vx', 'Vy', 'Vz',
'Cd', 'Cr', 'Drag Area', 'Sun Area', 'Mass', 'Orb Per Calc', 'Gaussian Quad', 'Flux Sigma Level', 'SolarFluxFile', 'Density Model', '2nd Order Oblateness'

Don't vary classical orbital elements and the cartesian state at the same time.

If 'Drag Area' is varied, the 'Sun Area' can be set to the same value so these are effectively varied together.

To vary the 'Flux Sigma Level' with runHPOP set to True requires the SolFlx_CSSI_Minus1.dat, SolFlx_CSSI_Minus2.dat, SolFlx_CSSI_Plus1.dat, SolFlx_CSSI_Plus2.dat files to be placed in C:\ProgramData\AGI\STK 11 (x64)\DynamicEarthData

Select how to vary:

*GridSearch*

Perform a full factorial grid search by varying the specified columns. Provide discrete values for each varied column.

*LatinHypercube*

Samples according to a latin hypercube, which leads to near uniform distribution along each of the varies axes except towards the edges of the upper and lower bound. Provide the lower and upper bounds for the varied columns. Provide the discrete values for 'SolarFluxFile', 'Density Model', '2nd Order Oblateness' which will be selected with equal probability.

*Perturb*

Perturb the nominal state or spacecraft properties according to a normal distribution. Provide the 1-$\sigma$ level for the varied columns. The following cannot be perturbed: 'Orb Per Calc', 'Gaussian Quad', 'Flux Sigma Level', 'SolarFluxFile', 'Density Model', '2nd Order Oblateness'




In [4]:
# # Example varying orbital elements
tradeStudy.howToVary = 'GridSearch'
tradeStudy.varyCols = ['a','e','i','AoP','RAAN']
tradeStudy.varyValues = [np.arange(6778,6978,100),
          np.arange(0,0.003,0.001),
          np.arange(0,135,45),
          np.arange(0,360,180),
          np.arange(0,360,180)]
tradeStudy.numberOfRuns = np.prod([len(values) for values in tradeStudy.varyValues])    

# # Example varying LT tradeStudyuration 
# tradeStudy.howToVary = 'GridSearch'
# tradeStudy.varyCols = ['Orb Per Calc','Gaussian Quad','Flux Sigma Level','SolarFluxFile','Density Model','2nd Order Oblateness']
# tradeStudy.varyValues =[np.array([1,10,50]),
#          np.array([1,10,50]),
#          np.array([-3,-2,-1,0,1,2,3]),
#          np.array(['SolFlx_CSSI.dat','SpaceWeather-v1.2.txt']),
#          np.array(['1976 Standard','Harris-Priester','Jacchia 1970','Jacchia 1971','Jacchia-Roberts','CIRA 1972','DTM 2012','NRLMSISE 2000','MSISE 1990','MSIS 1986','Jacchia70Lifetime']),
#          np.array(['On','Off'])]
# tradeStudy.numberOfRuns = np.prod([len(values) for values in tradeStudy.varyValues])    

# # Example sampling many LEOs with different s/c properties and lifetime tradeStudyurations   
# tradeStudy.howToVary = 'latinhypercube'
# tradeStudy.numberOfRuns = 200
# tradeStudy.varyCols = ['Rp','Ra','i','AoP','RAAN','TA',
#           'Cd','Cr','Drag Area','Mass',
#           'Orb Per Calc','Gaussian Quad','Flux Sigma Level','SolarFluxFile','Density Model','2nd Order Oblateness']
# tradeStudy.varyValues = [np.array([6778,7178]),
#             np.array([6778,7178]),
#             np.array([0,180]),
#             np.array([0,360]),
#             np.array([0,360]),
#             np.array([0,360]),
#             np.array([1.5,2.5]),
#             np.array([0.5,1.5]),
#             np.array([0.1,200]),
#             np.array([1,2000]),
#             np.array([1,20]),
#             np.array([1,20]),
#             np.array([-2,2]),
#             np.array(['SolFlx_CSSI.dat','SpaceWeather-v1.2.txt']),
#             np.array(['Jacchia 1970','Jacchia 1971','Jacchia-Roberts','NRLMSISE 2000','MSISE 1990','MSIS 1986']),
#             np.array(['On','Off'])]
# tradeStudy.setSunAreaEqualToDragArea = True
# tradeStudy.runHPOP = False                 
                        
# # Example of perturbing the cartesian state from a normal distribution
# tradeStudy.howToVary='Perturb'
# tradeStudy.numCores = 2
# tradeStudy.numberOfRuns = 10
# tradeStudy.varyCols = ['epoch','x','y','z','Vx','Vy','Vz','Cd','Cr','Drag Area','Mass']
# tradeStudy.varyValues = [30,10,10,10,0.01,0.01,0.01,0.1,0.05,1,1]

**Save the Trade Study**

In [5]:
saveTradeStudy(tradeStudy)
tradeStudy.properties()

{'fileName': 'LifeTimeExample.csv',
 'numCores': 2,
 'runHPOP': False,
 'maxDur': 100,
 'decayAlt': 65,
 'epoch': 19253.16666667,
 'a': 6778,
 'e': 0,
 'i': 45,
 'AoP': 0,
 'RAAN': 0,
 'TA': 0,
 'Cd': 1.0,
 'Cr': 1.0,
 'DragArea': 13.65,
 'SunArea': 15.43,
 'Mass': 1000,
 'AtmDen': 'Jacchia 1971',
 'SolFlxFile': 'SolFlx_CSSI.dat',
 'SigLvl': 0,
 'OrbPerCal': 10,
 'GaussQuad': 2,
 'SecondOrderOblateness': 'Off',
 'howToVary': 'GridSearch',
 'numberOfRuns': 72,
 'varyCols': ['a', 'e', 'i', 'AoP', 'RAAN'],
 'varyValues': [array([6778, 6878]),
  array([0.   , 0.001, 0.002]),
  array([ 0, 45, 90]),
  array([  0, 180]),
  array([  0, 180])],
 'setSunAreaEqualToDragArea': True}

## Create Runs

Generate a pandas DataFrame from the trade study file

*Special Case: Compare Lifetime Settings to HPOP*

A previously run trade study with HPOP results can be used to create a new trade study with different permutations of Lifetime Tool settings.

In [6]:
if not os.path.isfile(os.getcwd()+'\\Results\\'+ tradeStudy.fileName):
    df = generateTradeStudy(tradeStudy);
     
# # Various permutations of lifetime settings are used to compare to HPOP.
# tradeStudy = TradeStudy(fileName='LTVariationsVsHPOPLEOExtra.csv',howToVary='LifetimeVariations',numCores=4)
# tradeStudy.varyCols = ['Orb Per Calc','Gaussian Quad','2nd Order Oblateness']
# tradeStudy.varyValues =[np.array([1,5,10]),
#          np.array([1,2,8]),
#          np.array(['On','Off'])]
# previousHPOPRuns = 'LifeTimeHPOPComparisonExtra.csv' # 'LifeTimeHPOPComparison.csv'
# saveTradeStudy(tradeStudy)
# if not os.path.isfile(os.getcwd()+'\\Results\\'+ tradeStudy.fileName):
#     df = lifetimeVariations(os.getcwd()+'\\Results\\'+previousHPOPRuns,tradeStudy) # Assumes the previously run csv with HPOP results exists

## Load Trade Study Files and Results

In [7]:
tradeStudy = loadTradeStudy(tradeStudy.fileName.split('.')[0])
df = readResults(tradeStudy)
df

,Run ID,epoch,a,e,i,RAAN,AoP,TA,Rp,Ra,p,x,y,z,Vx,Vy,Vz,Cd,Cr,Drag Area,Sun Area,Mass,Cd*Drag Area/Mass,Cr*Sun Area/Mass,Orb Per Calc,Gaussian Quad,Flux Sigma Level,SolarFluxFile,Density Model,2nd Order Oblateness,LT Orbits,LT Years,LT Runtime
0,0.0,19253.166667,6778.0,0.000,0.0,360.0,360.0,360.0,6778.000,6778.000,6778.000000,6778.000,-0.0,0.0,0.0,7.668636,0.000000,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
1,1.0,19253.166667,6878.0,0.000,0.0,360.0,360.0,360.0,6878.000,6878.000,6878.000000,6878.000,-0.0,0.0,0.0,7.612684,0.000000,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
2,2.0,19253.166667,6778.0,0.001,0.0,360.0,360.0,360.0,6771.222,6784.778,6777.993222,6771.222,-0.0,0.0,0.0,7.676308,0.000000,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
3,3.0,19253.166667,6878.0,0.001,0.0,360.0,360.0,360.0,6871.122,6884.878,6877.993122,6871.122,-0.0,0.0,0.0,7.620300,0.000000,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
4,4.0,19253.166667,6778.0,0.002,0.0,360.0,360.0,360.0,6764.444,6791.556,6777.972888,6764.444,-0.0,0.0,0.0,7.683988,0.000000,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,67.0,19253.166667,6878.0,0.000,90.0,180.0,180.0,360.0,6878.000,6878.000,6878.000000,6878.000,-0.0,0.0,0.0,0.000000,-7.612684,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
68,68.0,19253.166667,6778.0,0.001,90.0,180.0,180.0,360.0,6771.222,6784.778,6777.993222,6771.222,-0.0,0.0,0.0,0.000000,-7.676308,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
69,69.0,19253.166667,6878.0,0.001,90.0,180.0,180.0,360.0,6871.122,6884.878,6877.993122,6871.122,-0.0,0.0,0.0,0.000000,-7.620300,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN
70,70.0,19253.166667,6778.0,0.002,90.0,180.0,180.0,360.0,6764.444,6791.556,6777.972888,6764.444,-0.0,0.0,0.0,0.000000,-7.683988,1.0,1.0,13.65,15.43,1000.0,0.01365,0.01543,10.0,2.0,0.0,SolFlx_CSSI.dat,Jacchia 1971,Off,NaN,NaN,NaN


## Run STK

Once the set of runs is ready in the pandas DataFrame, STK can setup to run and estimate the satellite lifetimes.

The cell below actually starts the computations. Below is some information on what to expect.

Since Jupyter Notebook uses asyncrio to run, the following cell will error when starting up STK, BUT STK is still being spun up on the specified number of threads. However, this also means that the cell will finish executing immediately and allow you to move on to other cells. If you execute other cells Jupyter will print any output beneath the most recently executed cell, so it is generally recommended to just wait until the runs are done. 

You can confirm that STK is being launched when you see this:

MainThread run_blocking_tasks: starting
MainThread run_blocking_tasks: creating executor tasks
MainThread run_blocking_tasks: waiting for executor tasks

Followed shortly by:

LifeTimeAnalysis0 has started.
LifeTimeAnalysis1 has started.
LifeTimeAnalysis2 has started.
etc.

These processes will appear as background processes in the Task Manager as AgUiApplications.

Once confirming everything appears to be running correctly, it may be best to work on something else or go take that coffee break!  (Unless your run size is small)

As the computations are performed the number of runs and runtime will be printed. The results will be saved to seperate csv files after a number of iterations or in the case of HPOP after each iteration.

Example: LifeTimeAnalysis0 5 of 1482. Total Time: 16.6
...
LifeTimeAnalysis0 wrote to LifeTimeResultsTestParallel0.csv

Sometimes if the computer goes to sleep or something goes awry only some of the results will be saved. In this case you may want to close the STK instances and rerun this script, which will merge any saved results into one file. If the script does run to completion sucessfully you will see:
'done' 'done 'done'

In [8]:
# Run
showSTK = False
saveEveryNIter = 20
# Check to see if data needs to be run
if tradeStudy.runHPOP == True:
    numOfMissingResults = len(df.loc[df['HPOP Years'].isna(),:])
else:
    numOfMissingResults = len(df.loc[df['LT Years'].isna(),:])
    
if numOfMissingResults != 0: 
    with suppress(Exception): # Used to ignore the "RuntimeError: Cannot close a running event loop" due to running in Jupyter
        df = df.sample(frac=1)
        main(df, tradeStudy,showSTK,saveEveryNIter)
else:
    print(tradeStudy.fileName+' has results for every run. Trade study is complete.')

MainThread run_blocking_tasks: starting
MainThread run_blocking_tasks: creating executor tasks
MainThread run_blocking_tasks: waiting for executor tasks


LifeTimeAnalysis0 has started.
LifeTimeAnalysis0 5 of 36. Total Time: 19.717957735061646
LifeTimeAnalysis0 10 of 36. Total Time: 20.81905770301819
LifeTimeAnalysis0 15 of 36. Total Time: 21.945292234420776
LifeTimeAnalysis0 20 of 36. Total Time: 22.881978034973145
	
LifeTimeAnalysis0 wrote to C:\Users\aclaybrook\Documents\PythonScripts\LifeTimeAnalysis\Results\LifeTimeExample0.csv
	
LifeTimeAnalysis0 25 of 36. Total Time: 24.040998697280884
LifeTimeAnalysis0 30 of 36. Total Time: 25.229782104492188
LifeTimeAnalysis0 35 of 36. Total Time: 26.38329267501831
LifeTimeAnalysis1 has started.
	
LifeTimeAnalysis0 done. Wrote to C:\Users\aclaybrook\Documents\PythonScripts\LifeTimeAnalysis\Results\LifeTimeExample0.csv
	
LifeTimeAnalysis1 5 of 36. Total Time: 30.891048431396484
LifeTimeAnalysis1 10 of 36. Total Time: 32.05337738990784
LifeTimeAnalysis1 15 of 36. Total Time: 33.059231996536255
LifeTimeAnalysis1 20 of 36. Total Time: 33.96885657310486
	
LifeTimeAnalysis1 wrote to C:\Users\aclaybroo

MainThread run_blocking_tasks: results: ['0 Done', '1 Done']
MainThread run_blocking_tasks: exiting


	
LifeTimeAnalysis1 done. Wrote to C:\Users\aclaybrook\Documents\PythonScripts\LifeTimeAnalysis\Results\LifeTimeExample1.csv
	


In [ ]:
# # View a specific trade study subset, useful for investigating which each STK instance is tasked with 
# kk = 0
# indPerDF = np.array_split(df.reset_index(drop=True).index,tradeStudy.numCores)
# dfSlice = df.iloc[indPerDF[kk]]
# dfSlice

Once the runs are finished, rerun this script. It will merge the csvs from multiple files into one csv.

If there are rows which did not execute the script will rerun those rows.

Once your lifetime analysis is done, you can start to investigate the results. Take a look at the LifeTimeAnalysisResults.ipynb to see examples of common types of lifetime analyses.